## Preambule

In [2]:
import numpy as np
import pandas as pd
import xarray as xr

# Plotting
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.colors import n_colors

## Input

In [3]:
rules_b = ['GF', 'PC', 'PCC', 'AP', 'GDR', 'ECPC']

## Read data

In [4]:
xr_dataread = xr.open_dataset("K:/ECEMF/T5.2/xr_dataread.nc")
#xr_2030 = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_alloc_2030.nc")

In [5]:
df_eng = pd.read_csv('X:/user/dekkerm/Data/ENGAGE/PolicyScenarios/engage-internal_snapshot_1695642798.csv')
df_eng = df_eng[df_eng.Variable == 'Emissions|Kyoto Gases']
df_eng = df_eng.reset_index(drop=True)
regions_df = np.array(df_eng.Region)
regions_df[regions_df == "Argentine Republic"] = 'ARG'
regions_df[regions_df == "Canada"] = 'CAN'
regions_df[regions_df == "Commonwealth of Australia"] = 'AUS'
regions_df[regions_df == "Federative Republic of Brazil"] = 'BRA'
regions_df[regions_df == "People's Repulic of China"] = 'CHN'
regions_df[regions_df == "European Union (28 member countries)"] = 'EU'
regions_df[regions_df == "Republic of India"] = 'IND'
regions_df[regions_df == "Republic of Indonesia"] = 'IDN'
regions_df[regions_df == "State of Japan"] = 'JPN'
regions_df[regions_df == "Russian Federation"] = 'RUS'
regions_df[regions_df == "Kingdom of Saudi Arabia"] = 'SAU'
regions_df[regions_df == "Republic of South Africa"] = 'ZAF'
regions_df[regions_df == "Republic of Korea (South Korea)"] = 'KOR'
regions_df[regions_df == "United Mexican States"] = 'MEX'
regions_df[regions_df == "Republic of Turkey"] = 'TUR'
regions_df[regions_df == "United States of America"] = 'USA'
regions_df[regions_df == "Viet Nam "] = 'VNM'
df_eng.Region = regions_df

In [6]:
curpol = "GP_CurPol_T45"
ndc = "GP_NDC2030_T45"
nz = "GP_Glasgow"
df_eng_ref = df_eng[['Model', 'Scenario', 'Region']+list(df_eng.keys()[5:])]
df_eng_ref = df_eng_ref[df_eng_ref.Scenario.isin([curpol, ndc, nz])]
scen = np.array(df_eng_ref.Scenario)
scen[scen == ndc] = 'NDC'
scen[scen == curpol] = 'CurPol'
scen[scen == nz] = 'NetZero'
reg = np.array(df_eng_ref.Region)
reg[reg == 'World'] = 'WORLD'
df_eng_ref['Scenario'] = scen
df_eng_ref['Region'] = reg
dummy = df_eng_ref.melt(id_vars=["Scenario", "Model", "Region"], var_name="Time", value_name="Value")
dummy['Time'] = np.array(dummy['Time'].astype(int))
dummy = dummy.set_index(["Scenario", "Model", "Region", "Time"])
xr_eng = xr.Dataset.from_dataframe(dummy)
xr_eng = xr_eng.reindex(Time = np.arange(1850, 2101))
xr_eng = xr_eng.interpolate_na(dim="Time", method="linear")

## Total budgets - globe

In [21]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("1.5&deg;C (50%)", "1.7&deg;C (50%)", "2.0&deg;C (50%)"),
                    horizontal_spacing = 0.02, vertical_spacing=0.06, shared_yaxes=True)

for col_i in range(3):
    risk = [0.5]
    temp = [1.5, 1.7, 2.0][col_i]

    col = 'silver'
    allpars = ['Risk', 'NegEmis', 'TrajUnc']

    dataset = xr_dataread.GHG_globe.sel(Temperature=temp, Risk=risk, Time=np.arange(2021, 2101))/1e3
    fig.add_trace(go.Scatter(x=dataset.Time,
                            y=dataset.min(dim=allpars),
                            name='SSP-baselines',
                            line={'color': col, 'width': 1},
                            mode='lines',
                            fill=None,
                            showlegend=False), 1, 1+col_i)
    fig.add_trace(go.Scatter(x=dataset.Time,
                            y=dataset.max(dim=allpars),
                            name='All pathways (1.5&deg;C, 50%)',
                            line={'color': col, 'width': 1},
                            mode='lines',
                            fill='tonexty',
                            showlegend=False), 1, 1+col_i)

    allpars = ['NegEmis', 'Risk']
    dataset = xr_dataread.GHG_globe.sel(Temperature=temp, Risk=risk, Time=np.arange(2021, 2101), TrajUnc='Earliest', NegEmis=[0.2, 0.3, 0.4, 0.5])/1e3
    fig.add_trace(go.Scatter(x=dataset.Time,
                            y=dataset.min(dim=allpars),
                            name='SSP-baselines',
                            line={'color': 'steelblue', 'width': 2},
                            mode='lines',
                            fill=None,
                            showlegend=False), 1, 1+col_i)
    fig.add_trace(go.Scatter(x=dataset.Time,
                            y=dataset.max(dim=allpars),
                            name='Early action, less negative emissions',
                            line={'color': 'steelblue', 'width': 2},
                            mode='lines',
                            fill='tonexty',
                            showlegend=False), 1, 1+col_i)

    dataset = xr_dataread.GHG_globe.sel(Temperature=temp, Risk=risk, Time=np.arange(2021, 2101), TrajUnc='Latest', NegEmis=[0.5, 0.6, 0.7, 0.8])/1e3
    fig.add_trace(go.Scatter(x=dataset.Time,
                            y=dataset.min(dim=allpars),
                            name='SSP-baselines',
                            line={'color': 'tomato', 'width': 2},
                            mode='lines',
                            fill=None,
                            showlegend=False), 1, 1+col_i)
    fig.add_trace(go.Scatter(x=dataset.Time,
                            y=dataset.max(dim=allpars),
                            name='Late action, more negative emissions',
                            line={'color': 'tomato', 'width': 2},
                            mode='lines',
                            fill='tonexty',
                            showlegend=False), 1, 1+col_i)

                            
    dataset = xr_dataread.GHG_globe.sel(Temperature=temp, Risk=risk, Time=2030)/1e3
    fig.add_trace(go.Scatter(x=[2030, 2030],
                            y=[np.min(dataset), np.max(dataset)],
                            line={'color': "black", 'width': 3},
                            marker={'size': 17},
                            mode='lines+markers',
                            fill=None,
                            showlegend=False), 1, 1+col_i)
                            
    dataset = xr_dataread.GHG_globe.sel(Temperature=temp, Risk=risk, Time=2040)/1e3
    fig.add_trace(go.Scatter(x=[2040, 2040],
                            y=[np.min(dataset), np.max(dataset)],
                            line={'color': "black", 'width': 3},
                            marker={'size': 17},
                            mode='lines+markers',
                            fill=None,
                            showlegend=False), 1, 1+col_i)
    fig.add_vline(x=2030, line_dash="dot", row=1, col=1+col_i, line_color="black", line_width=2)
    fig.add_vline(x=2040, line_dash="dot", row=1, col=1+col_i, line_color="black", line_width=2)
    fig.add_hline(y=0, line_dash="solid", row=1, col=1+col_i, line_color="black", line_width=2)
    if col_i == 0: fig.update_yaxes(tickfont=dict(size=20), row=1, col=1+col_i, title='Global GHG emissions (Gt CO<sub>2</sub>e/yr)', titlefont=(dict(size=25)))
    else: fig.update_yaxes(tickfont=dict(size=20), row=1, col=1+col_i)
    fig.update_xaxes(tickfont=dict(size=20), row=1, col=1+col_i)
fig.update_annotations(font_size=25)
fig.update_layout(height=700, width=2000, template='simple_white')
#fig.update_layout(legend=dict(x=1, y=1, xanchor='right', yanchor='top', font=dict(size=25)))
#fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/GlobalPathways_empty.png", scale=3)
fig.write_image("X:/user/dekkerm/Projects/ECEMF_T5.2/Figures/Presentations/GlobalPathways_filled.png", scale=3)
#fig.show()